In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series

path = "refFlat.txt"
path_fa = "result.fa"

In [2]:
def read_fasta(file_path):
    sequences = {}
    seq_id = None
    current_seq = ""

    with open(file_path, "r") as file:
        for line in file:
            line = line.strip()

            if line.startswith(">"):  
                if seq_id is not None:
                    sequences[seq_id] = current_seq
                    current_seq = ""
                seq_id = line[1:]  
            else:
                current_seq += line

        if seq_id is not None and current_seq != "":
            sequences[seq_id] = current_seq

    return sequences



In [3]:
import re

def split_by_space(text):
    return re.split('\s+', text)

In [4]:
def gene_data(text_list):
    contents =[]
    for i in range(len(text_list)-1):
        contents.append(text_list[i])
    return contents

In [5]:
def result(path):
    with open(path, "r", encoding="utf-8") as file:
        lines =file.readlines()
    result = []
    for line in lines:
        line =split_by_space(line)
        contents=gene_data(line)
        result.append(contents)
    return result
    

In [6]:
gene_data =result(path)
col_names =['geneName','name','chrom','strand','txStart','txEnd','cdsStart','cdsEnd','exonCount','exonStarts','exonEnds']
gene_df=pd.DataFrame(gene_data,columns=col_names)

In [7]:
base_data = read_fasta(path_fa)

In [8]:
df=gene_df[gene_df.duplicated('name', keep=False)]

In [9]:
def target_base_CT_plus(seq):
    match = re.findall('(?=(\w{20}GG))', seq)
    target_list=[]
    for i in range(len(match)):
        target = match[i][0:3]
        if target =='CAA'or target =='CAG'or target =='CGA':
            target_list.append(True)
        
        else:
            target_list.append(False)
    result =[match[s] for s in range(len(match)) if target_list[s]]
    result = list(dict.fromkeys(result))
    return result

In [10]:
def target_base_AG_plsu(seq):
    match = re.findall('(?=(CC\w{20}))', seq)
    target_list=[]
    print(match)
    for i in range(len(match)):
        target = match[i][-3:]
        print(target)
        if target =='TGG':
            target_list.append(True)
        
        else:
            target_list.append(False)
    result =[match[s] for s in range(len(match)) if target_list[s]]
    result = list(dict.fromkeys(result))
    return result

In [11]:
def target_start(target_list,qua):    
    result = []
    for i in target_list:
        matches = re.finditer(i, qua)
        for match in matches:
            start_index = match.start()
            result.append(start_index)
    return result

In [12]:
def target_gene_start(name,target):
    data = gene_df[gene_df['name']==name]
    num = data['txStart'].astype(int).to_numpy()
    target = np.array(target)
    print(num)
    print(target)
    result = target+num
    #ここ-1が必要かもしれない？
    return result
    

In [13]:
samp = base_data["NM_001385633::chr7:44239875-44254175"]

In [14]:
target_gene_start("NM_001385633",target_start(target_base_CT_plus(samp),samp))

[44239875]
[  940  1010  1125  1492  1937  2813  2910  3599  3607  3939  4182  4259
  4483  4546  4811  5806  5868  5947  6648  6688  6747  6757  6854  7045
  7050  7082  7926  8028  8424  8617  8770  9406  9714 10339 11442 11556
 11674 12151 13084 13360 14021 14077 14116 14120]


array([44240815, 44240885, 44241000, 44241367, 44241812, 44242688,
       44242785, 44243474, 44243482, 44243814, 44244057, 44244134,
       44244358, 44244421, 44244686, 44245681, 44245743, 44245822,
       44246523, 44246563, 44246622, 44246632, 44246729, 44246920,
       44246925, 44246957, 44247801, 44247903, 44248299, 44248492,
       44248645, 44249281, 44249589, 44250214, 44251317, 44251431,
       44251549, 44252026, 44252959, 44253235, 44253896, 44253952,
       44253991, 44253995])

In [15]:
target_list= target_base_CT_plus(samp)

In [16]:
target_base_CT_plus(samp)

['CGAGGAGGAGCTGGGCTTCTGG',
 'CAGCACCGTGACGCTGAGGAGG',
 'CAGGAGGCGGCGGCCTGGACGG',
 'CGAAAACTTAAGCGTCCCTGGG',
 'CAAACTACTGGTCCTAGTCAGG',
 'CAGTGCCGCATTCGTACTGCGG',
 'CGACATTTGGGGTTGAGTTGGG',
 'CAGTGCTGCAGCCTGCAGGTGG',
 'CAGCCTGCAGGTGGGGGAGGGG',
 'CAGTGGGATACTGACAATGAGG',
 'CAAACCCAGGTTTTTGAGCCGG',
 'CAAAATATATAAAGAGGGCTGG',
 'CAGAAAGGGCTCTTTCCCTGGG',
 'CAGAACTGATTGCTAGAAGTGG',
 'CAGGCCTCCCCAATCCCTGGGG',
 'CAACAGGGCAGGTGAGGCTGGG',
 'CAGGGTTGGAGGGGATGCAGGG',
 'CAAGTGGCTAGAAGGCTGGAGG',
 'CGACTACTAGGAAGGCTGGAGG',
 'CAGTTTGTGACTGTGGCCGGGG',
 'CAAGTGGAGTCAGGCAGCCTGG',
 'CAGGCAGCCTGGAGGTTAGAGG',
 'CAGGGTCTGCAGAGGGCAGCGG',
 'CAGGCCAGGAAGAGGAGGGAGG',
 'CAGGAAGAGGAGGGAGGAGAGG',
 'CAGTTGCGCCTGATTTCTCGGG',
 'CAGTAGACCCCCGTCCCAATGG',
 'CAATCACTCCCGGAAGCCGGGG',
 'CAACGCCAACGCAGCAGCCTGG',
 'CGACTCCCACTGTCACCCCAGG',
 'CAGTCACCAAGGTGGTTTATGG',
 'CAGAGGCTGAACACACTGGTGG',
 'CAGGTGTGAGAGAAAGCCTAGG',
 'CAACAGTGAGCCTGTGTTGGGG',
 'CAGCATTGTGGGCCTGAGCGGG',
 'CAGCCCTCCCCCCTGTGGGGGG',
 'CGATGCCAGCAGACACCCCCGG',
 

In [17]:
target_start(target_base_CT_plus(samp),samp)

[940,
 1010,
 1125,
 1492,
 1937,
 2813,
 2910,
 3599,
 3607,
 3939,
 4182,
 4259,
 4483,
 4546,
 4811,
 5806,
 5868,
 5947,
 6648,
 6688,
 6747,
 6757,
 6854,
 7045,
 7050,
 7082,
 7926,
 8028,
 8424,
 8617,
 8770,
 9406,
 9714,
 10339,
 11442,
 11556,
 11674,
 12151,
 13084,
 13360,
 14021,
 14077,
 14116,
 14120]